In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
# You will need to install NLTK for the next steps
# pip install nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df_path = '/content/drive/MyDrive/radiology.csv'
df = pd.read_csv(df_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(f"Total number of notes loaded: {len(df)}")
print("\nFirst 5 rows of data:")
print(df.head())
print("\nColumn data types:")
print(df.info())

Total number of notes loaded: 2321355

First 5 rows of data:
          note_id  subject_id     hadm_id note_type  note_seq  \
0  10000032-RR-14    10000032  22595853.0        RR        14   
1  10000032-RR-15    10000032  22595853.0        RR        15   
2  10000032-RR-16    10000032  22595853.0        RR        16   
3  10000032-RR-18    10000032         NaN        RR        18   
4  10000032-RR-20    10000032         NaN        RR        20   

             charttime            storetime  \
0  2180-05-06 21:19:00  2180-05-06 23:32:00   
1  2180-05-06 23:00:00  2180-05-06 23:26:00   
2  2180-05-07 09:55:00  2180-05-07 11:15:00   
3  2180-06-03 12:46:00  2180-06-03 14:01:00   
4  2180-07-08 13:18:00  2180-07-08 14:15:00   

                                                text  
0  EXAMINATION:  CHEST (PA AND LAT)\n\nINDICATION...  
1  EXAMINATION:  LIVER OR GALLBLADDER US (SINGLE ...  
2  INDICATION:  ___ HCV cirrhosis c/b ascites, hi...  
3  EXAMINATION:  Ultrasound-guided paracentes

In [ ]:
# --- 1. Data Type Correction ---
# Convert time columns to datetime objects
df['charttime'] = pd.to_datetime(df['charttime'])
df['storetime'] = pd.to_datetime(df['storetime'])

# Convert hadm_id to a nullable integer type (Int64) since it has missing values (NaN)
df['hadm_id'] = df['hadm_id'].astype('Int64')

# --- 2. Structural Analysis: Calculate Length and Word Count ---
df['char_count'] = df['text'].astype(str).apply(len)
df['word_count'] = df['text'].astype(str).apply(lambda x: len(x.split()))

# Calculate Time Delay: time between chart and store
df['time_delay'] = df['storetime'] - df['charttime']

# --- 3. Descriptive Statistics ---
print("Descriptive Statistics for Numerical and Structural Features:")
print(df[['char_count', 'word_count', 'note_seq']].describe())

print("\nDescriptive Statistics for Time Delay:")
print(df['time_delay'].describe())

Descriptive Statistics for Numerical and Structural Features:
         char_count    word_count      note_seq
count  2.321355e+06  2.321355e+06  2.321355e+06
mean   1.158875e+03  1.687388e+02  4.396245e+01
std    1.004701e+03  1.486386e+02  4.312100e+01
min    3.000000e+00  1.000000e+00  2.000000e+00
25%    5.110000e+02  7.300000e+01  1.800000e+01
50%    7.810000e+02  1.120000e+02  3.000000e+01
75%    1.440000e+03  2.100000e+02  5.500000e+01
max    3.584900e+04  3.081000e+03  8.590000e+02

Descriptive Statistics for Time Delay:
count                      2321355
mean     0 days 08:14:48.686056204
std      2 days 00:54:43.509875263
min            -844 days +10:49:00
25%                0 days 01:07:00
50%                0 days 02:15:00
75%                0 days 05:07:00
max              661 days 21:13:00
Name: time_delay, dtype: object


In [ ]:
# --- Investigate Negative Delays ---
# How many notes have a negative time delay?
negative_delay_count = (df['time_delay'] < pd.Timedelta(seconds=0)).sum()
print(f"Number of notes with negative time delay: {negative_delay_count}")

# --- Decide on Handling Strategy ---
# Strategy 1: Remove impossible records (recommended if the number is small)
# df_clean = df[df['time_delay'] >= pd.Timedelta(seconds=0)].copy()

# Strategy 2: Cap the impossible values at zero (recommended if the number is large)
df['time_delay_clean'] = df['time_delay'].apply(
    lambda x: pd.Timedelta(seconds=0) if x < pd.Timedelta(seconds=0) else x
)

Number of notes with negative time delay: 652


In [ ]:
# Create a mask for valid time delays (non-negative)
valid_mask = df['time_delay'] >= pd.Timedelta(seconds=0)

# Filter the DataFrame to keep only valid records
df_clean = df[valid_mask].copy()

print(f"Original note count: {len(df)}")
print(f"Cleaned note count: {len(df_clean)}")

# You should use df_clean for all subsequent steps!

Original note count: 2321355
Cleaned note count: 2320703


In [ ]:
# --- Aggregation and Descriptive Statistics per Patient ---

# Group by subject_id and aggregate the desired features
patient_summary = df_clean.groupby('subject_id').agg(
    notes_per_patient=('note_id', 'count'), # Count of notes per patient
    mean_char_count=('char_count', 'mean'),
    std_char_count=('char_count', 'std'),
    max_char_count=('char_count', 'max'),
    mean_word_count=('word_count', 'mean'),
    std_word_count=('word_count', 'std'),
    max_word_count=('word_count', 'max')
)

# Calculate descriptive statistics on the aggregated table
print("Descriptive Statistics for Structural Features Aggregated by Patient (Subject ID):")
print("\nNotes Per Patient Statistics:")
print(patient_summary['notes_per_patient'].describe())

print("\nMean Word Count Per Patient Statistics:")
print(patient_summary['mean_word_count'].describe())

print("\nMaximum Word Count Per Patient Statistics:")
print(patient_summary['max_word_count'].describe())

Descriptive Statistics for Structural Features Aggregated by Patient (Subject ID):

Notes Per Patient Statistics:
count    237408.000000
mean          9.775168
std          15.837470
min           1.000000
25%           2.000000
50%           4.000000
75%          11.000000
max         428.000000
Name: notes_per_patient, dtype: float64

Mean Word Count Per Patient Statistics:
count    237408.000000
mean        155.334841
std          89.557641
min           9.000000
25%          94.500000
50%         136.166667
75%         192.000000
max        1790.000000
Name: mean_word_count, dtype: float64

Maximum Word Count Per Patient Statistics:
count    237408.000000
mean        313.101884
std         230.597792
min           9.000000
25%         127.000000
50%         257.000000
75%         447.000000
max        3081.000000
Name: max_word_count, dtype: float64


In [ ]:
# --- Cell 1: Setup and Function Definition ---
import nltk
import re
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
import pandas as pd

# --- DEPENDENCY SETUP ---
# Running the download ensures the resource is available.
try:
    nltk.download('stopwords', quiet=True)
except LookupError:
    print("NLTK download failed.")

# Set up stopwords AFTER download
custom_stopwords = set(stopwords.words('english'))

# --- 1. Extraction Function ---
def extract_impression(note_text):
    text = str(note_text)
    impression_pattern = r'IMPRESSION\s*:?\s*(.*)'
    match = re.search(impression_pattern, text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return ""

# --- 2. Cleaning and Tokenization Function ---
def clean_and_tokenize(text):
    text = str(text).lower()

    # Remove punctuation and split into tokens
    tokens = re.findall(r'\b\w+\b', text)

    # Filter out stopwords and single-character tokens
    cleaned_tokens = [
        token for token in tokens
        if token not in custom_stopwords and len(token) > 1
    ]
    return cleaned_tokens

# --- 3. N-gram Analysis Function ---
def get_top_ngrams(tokens, n=2, top_k=20):
    n_grams = list(ngrams(tokens, n))
    n_gram_counts = Counter(n_grams)
    top_n_grams = [(' '.join(ng), count) for ng, count in n_gram_counts.most_common(top_k)]
    return pd.DataFrame(top_n_grams, columns=['Phrase', 'Count'])

print("Block 1: Setup and function definition complete.")

Block 1: Setup and function definition complete.


In [ ]:
!pip install tqdm

In [ ]:
!pip install swifter

In [ ]:
# --- Cell 2 (Final, Fully Optimized Block) ---
import swifter
from tqdm.auto import tqdm
from itertools import chain # <-- NEW IMPORT

tqdm.pandas()

# 1. Extract the Impression Text
print("Starting Impression Extraction...")
df_clean['impression_text'] = df_clean['text'].apply(extract_impression)

# 2. Generate Tokens (Parallel Processing)
# The output is a Series of lists, which should be very fast thanks to swifter
print("Starting tokenization (Parallel Processing)...")
token_series = df_clean['impression_text'].swifter.apply(clean_and_tokenize)
# The previous step is complete, now we perform the efficient aggregation:

# 3. Efficient Aggregation (Memory-Optimized)
print("Starting efficient token aggregation...")
# chain.from_iterable creates a fast generator to join the lists
all_impression_tokens = list(chain.from_iterable(token_series))

print("Block 2: Tokenization and Aggregation complete. Ready for analysis.")

Starting Impression Extraction...
Starting tokenization (Parallel Processing)...


Pandas Apply:   0%|          | 0/2320703 [00:00<?, ?it/s]

Starting efficient token aggregation...
Block 2: Tokenization and Aggregation complete. Ready for analysis.


In [ ]:
# --- Cell 3: Analysis and Results ---

# Top Unigrams (Single Words)
word_counts = Counter(all_impression_tokens)
top_unigrams = pd.DataFrame(word_counts.most_common(20), columns=['Word', 'Count'])
print("\nTop 20 Most Frequent Words (Unigrams) in IMPRESSION:")
print(top_unigrams)

# Top Bigrams (Two-Word Phrases)
top_bigrams_df = get_top_ngrams(all_impression_tokens, n=2, top_k=20)
print("\nTop 20 Most Frequent Two-Word Phrases (Bigrams) in IMPRESSION:")
print(top_bigrams_df)

print("\nBlock 3: Descriptive Analysis Complete.")


Top 20 Most Frequent Words (Unigrams) in IMPRESSION:
         Word   Count
0       right  941899
1         ___  904920
2        left  825315
3    evidence  558428
4       acute  457797
5    findings  363552
6       small  323036
7        mild  290497
8     pleural  287334
9   pulmonary  280518
10   fracture  278554
11      lower  233554
12     stable  219800
13  unchanged  219028
14       lobe  211884
15     normal  209737
16       seen  207313
17  bilateral  204119
18   effusion  203789
19    changes  199228

Top 20 Most Frequent Two-Word Phrases (Bigrams) in IMPRESSION:
                     Phrase   Count
0                    dr ___  166534
1          pleural effusion  156542
2                   ___ ___  130590
3     acute cardiopulmonary  101683
4           pulmonary edema   96395
5      degenerative changes   93145
6                lower lobe   93105
7                   bi rads   89372
8        findings discussed   85741
9                left lower   83817
10        pleural effusi